# Problem Set 4 - Structural Estimation

## John Wilson
### July 15, 2017

In [1]:
import numpy as np
import scipy.stats as sts
from scipy.special import gamma, beta
from matplotlib import pyplot as plt
import scipy.optimize as opt
import scipy.integrate as integrate
%matplotlib notebook

## Problem 2

### Part a)

In [2]:
data = np.genfromtxt('data/MacroSeries.txt', delimiter=',')
c = data[:,0]
k = data[:,1]
w = data[:,2]
r = data[:,3]

In [3]:
def zpdf(z, rho, mu, sigma):
    mean = rho * z[:-1] + (1 - rho) * mu
    mean = np.append(mu, mean)
    pdf_vals    = ((1/(sigma * np.sqrt(2 * np.pi)) *
                    np.exp( - (z - mean)**2 / (2 * sigma**2))))
    return pdf_vals

def log_lik_norm(alpha, rho, mu, sigma, w, k):
    z = np.log(w) - np.log(1 - alpha) - alpha * np.log(k)
    pdf_vals = zpdf(z, rho, mu, sigma)
    pdf_vals[np.isclose(pdf_vals, np.zeros_like(pdf_vals), atol=1e-25)] = 1e-24
    ln_pdf_vals = np.log(pdf_vals)
    return ln_pdf_vals.sum()

def crit(params, *args):
    alpha, rho, mu, sigma = params
    w, k = args
    return -1 * log_lik_norm(alpha, rho, mu, sigma, w, k)

In [4]:
#Define initial guesses for each parameter
alpha_init = .9
rho_init = 0.5
mu_init = 15
sigma_init = 15
params_init = np.array([alpha_init, rho_init, mu_init, sigma_init])
results = opt.minimize(crit, params_init, args=(w, k),
                       method = 'L-BFGS-B', bounds = ((1e-10, 1 - 1e-10), \
                                                      (-1 + 1e-10, 1 - 1e-10),\
                                                     (1e-10, None), (1e-10, None)))
                       
alpha_MLE, rho_MLE, mu_MLE, sigma_MLE = results.x
likelihood_MLE = -results.fun
print('alpha_MLE=', alpha_MLE, ' rho_MLE=', rho_MLE, ' mu_MLE=', \
      mu_MLE, ' sigma_MLE=', sigma_MLE)
print("Function value: ", likelihood_MLE)
print(results.hess_inv.todense())

alpha_MLE= 0.457484260467  rho_MLE= 0.720517391754  mu_MLE= 9.52316706261  sigma_MLE= 0.0919960982288
Function value:  96.7069080296
[[  5.98511631e+00  -5.59097550e+00  -8.43732369e+01  -8.16761076e-01]
 [ -5.59097550e+00   5.86626644e+00   7.99135120e+01   1.20481280e+00]
 [ -8.43732369e+01   7.99135120e+01   1.19133648e+03   1.22810578e+01]
 [ -8.16761076e-01   1.20481280e+00   1.22810578e+01   4.20200668e-01]]


### Part b)

In [5]:
def log_lik_norm_r(alpha, rho, mu, sigma, r, k):
    z = np.log(r) - np.log(alpha) - (alpha- 1) * np.log(k)
    pdf_vals = zpdf(z, rho, mu, sigma)
    pdf_vals[np.isclose(pdf_vals, np.zeros_like(pdf_vals), atol=1e-25)] = 1e-24
    ln_pdf_vals = np.log(pdf_vals)
    return ln_pdf_vals.sum()

def crit_r(params, *args):
    alpha, rho, mu, sigma = params
    r, k = args
    return -1 * log_lik_norm_r(alpha, rho, mu, sigma, r, k)

In [6]:
#Define initial guesses for each parameter
alpha_init = .5
rho_init = 1
mu_init = 15
sigma_init = 15
params_init = np.array([alpha_init, rho_init, mu_init, sigma_init])
results = opt.minimize(crit_r, params_init, args=(r, k),
                       method = 'L-BFGS-B', bounds = ((1e-10, 1 - 1e-10), \
                                                      (-1 + 1e-10, 1 - 1e-10),\
                                                     (1e-10, None), (1e-10, None)))
                       
alpha_MLE, rho_MLE, mu_MLE, sigma_MLE = results.x
likelihood_MLE = -results.fun
print('alpha_MLE=', alpha_MLE, ' rho_MLE=', rho_MLE, ' mu_MLE=', \
      mu_MLE, ' sigma_MLE=', sigma_MLE)
print("Function value: ", likelihood_MLE)
print(results.hess_inv.todense())

alpha_MLE= 0.457471361248  rho_MLE= 0.720517186549  mu_MLE= 9.37109997912  sigma_MLE= 0.0919969067217
Function value:  96.7069080025
[[  4.90897407e+01  -1.40790443e+02  -8.86624846e+02   9.13744126e-01]
 [ -1.40790443e+02   4.25011928e+02   2.54104974e+03  -4.59996527e+00]
 [ -8.86624846e+02   2.54104974e+03   1.60137585e+04  -1.63342458e+01]
 [  9.13744126e-01  -4.59996527e+00  -1.63342458e+01   2.02760706e-01]]


### Part c)

First, some basic calculations are done. Given that r=1, equation 4 implies that $z^* = \log (\alpha k_t^{\alpha - 1})$. So given the parameters obtained in parts a and b, we find the probability that $z > z^*$ given that $z_{t-1} = 10$.

In [10]:
kt = 7500000
z_prev = 10
mean = rho_MLE * z_prev + (1 - rho_MLE) * mu_MLE
var = sigma_MLE
z_star = -np.log(alpha_MLE) - (alpha_MLE - 1) * np.log(kt)
prob = 1 - sts.norm.cdf(z_star, mean, var)
print("The probability that interest rate is greater than one is {}.".format(prob))

The probability that interest rate is greater than one is 0.999999593284641.
